In [1]:
import time

import torch
from torch import nn
from torch.nn import functional as F


In [2]:
input = torch.rand(3,6,15000)
input.shape

torch.Size([3, 6, 15000])

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

class Conv_block(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels, 32, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels, 32, kernel_size=8, stride=2, padding=3)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat([x1, x2, x3], dim=1)
        x = self.relu(x)
        return x

class Chrononet(nn.Module):
    def __init__(self, in_channels=6):
        super().__init__()
        self.conv_block1 = Conv_block(6)
        self.conv_block2 = Conv_block(96)
        self.conv_block3 = Conv_block(96)
        self.gru1 = nn.GRU(96, 32, batch_first=True)
        self.gru2 = nn.GRU(32, 32, batch_first=True)
        self.gru3 = nn.GRU(64, 32, batch_first=True)
        self.linear = nn.Linear(1875*2, 1)
        self.gru4 = nn.GRU(96, 32, batch_first=True)
        self.last_lin = nn.Linear(32, 2)

    def forward(self, x, return_emb=False):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.permute(0, 2, 1)
        x1, _ = self.gru1(x)
        x2, _ = self.gru2(x1)
        x = torch.cat([x1, x2], dim=2)
        x3, _ = self.gru3(x)
        x = torch.cat([x1, x2, x3], dim=2)
        x4 = x.permute(0, 2, 1)
        x = self.linear(x4)
        x = F.relu(x)
        x = x.permute(0, 2, 1)
        x4, _ = self.gru4(x)
        x4 = x4.flatten(1, 2)
        out = self.last_lin(x4)
        if return_emb:
            return out, x4
        else:
            return out




In [4]:
import mne

In [5]:
true_numbers = ['1', '2','3','4','5', '6','7','8','9','10','11','12','13','14','15','26','27','28','29','30']
people_list = ['3', '4', '5', '7, 10', '12', '15', '26', '28', '29', '30', '16', '19', '20', '22', '25', '31', '32', '33', '34', '2', '6', '23', '27', '28', '8']
arr_of_ahi = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0], [56.5, 50.5],[15.2, 22.2],[10.1, 29.7],[22.2, 23.9],[40.0, 41.7],[1.83, 19.5],[37.5, 27.8],[14.9, 11.5],[78.2, 52.5],[20.1, 19.9],[0,0],[0,0],[0,0],[0,0],[0,0],[17.7, 13.2],[53.0, 47.6],[20.7, 21.4],[21.4, 31.4],[101.0, 36.4],[92.6, 125.0],[104.0, 92.0],[73.8, 82.8],[73.8, 64.1],[21.0, 19.9]]
arr = []
import numpy as np
def read_data(path):
    print(path)
    match = re.search(r'\d+', path)
    files = os.listdir(path)
    files = [os.path.join(path, f) for f in files if f.endswith(".EDF")]
    for file in files:
        print(file)
        datax=mne.io.read_raw_edf(file,preload=True)
        datax.set_eeg_reference()
        datax.filter(l_freq=1,h_freq=45)
        epochs=mne.make_fixed_length_epochs(datax,duration=150 ,overlap=0)
        epochs=epochs.get_data()
        arr.append(match.group())
        if match.group() in true_numbers:
            return [epochs.astype(np.float32), 0]
        else:
            return [epochs.astype(np.float32), 1]
        
def read_data_for_cb(path, s):
    print(path)
    match = re.search(r'\d+', path)
    files = os.listdir(path)
    files = [os.path.join(path, f) for f in files if f.endswith(".EDF")]
    for file in files:
        print(file)
        datax=mne.io.read_raw_edf(file,preload=True)
        datax.set_eeg_reference()
        datax.filter(l_freq=1,h_freq=45)
        epochs=mne.make_fixed_length_epochs(datax,duration=150 ,overlap=0)
        epochs=epochs.get_data()
        arr.append(match.group())
        if match.group() in true_numbers:
            return epochs.astype(np.float32), [s for i in range(epochs.shape[0])]
        else:
            return epochs.astype(np.float32), [s for i in range(epochs.shape[0])]

In [6]:
import re
from pathlib import Path
def execute(path):

    files = os.listdir(path)
    #print(files)
    paths1 = [os.path.join(path, f) for f in files if f.endswith(".REC") or f.endswith(".rec")]
    if len(paths1)==0:
        return 0
    old_filename = Path(paths1[0])

    new_extension = '.EDF'

    new_filename = old_filename.with_suffix(new_extension)
    try:
        old_filename.rename(new_filename)
    except:
        pass

In [7]:
import os
dirs = os.listdir()
dirs_filtered = [dir for dir in dirs if 'N' in dir][1:]
dirs_filtered.pop(8)
dirs_filtered_2 = []
X_for_cb = []
y_for_cb = []

for s in dirs_filtered:
  dirs_n = os.listdir(s)
  dirs_filtered_new = [s + "\\" + dir for dir in dirs_n if 'N' in dir]
  dirs_filtered_2.append(dirs_filtered_new)
data1 = []
data2 = []
for i in range(len(dirs_filtered_2)):
    for j in range(len(dirs_filtered_2[i])):
        execute(dirs_filtered_2[i][j])
        try:
            matches = re.findall(r'\d+', dirs_filtered_2[i][j])
            elem, metka = read_data_for_cb(dirs_filtered_2[i][j], arr_of_ahi[int(matches[0])][int(matches[1])])
            X_for_cb.append(elem)
            y_for_cb+=metka
            
            if matches[0] in people_list:
                data1.append(read_data(dirs_filtered_2[i][j]))
            else:
               data2.append(read_data(dirs_filtered_2[i][j]))
        except Exception as e:
            print(e)

Np 10\Nr 1
Np 10\Nr 1\N-1.EDF
Extracting EDF parameters from C:\Users\User\Downloads\sample\Np 10\Nr 1\N-1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5824799  =      0.000 ... 29123.995 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 661 samples (3.305 s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s


Not setting metadata
194 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 194 events and 30000 original time points ...


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.7s finished


0 bad epochs dropped
Np 10\Nr 1
Np 10\Nr 1\N-1.EDF
Extracting EDF parameters from C:\Users\User\Downloads\sample\Np 10\Nr 1\N-1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5824799  =      0.000 ... 29123.995 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 661 samples (3.305 s)


...

0 bad epochs dropped
Np 9\Nr 1
Np 9\Nr 1\N-1.EDF
Extracting EDF parameters from C:\Users\User\Downloads\sample\Np 9\Nr 1\N-1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5858999  =      0.000 ... 29294.995 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 661 samples (3.305 s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


Not setting metadata
195 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 195 events and 30000 original time points ...


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.6s finished


0 bad epochs dropped
list index out of range


In [8]:
print(dirs_filtered)

['Np 10', 'Np 11', 'Np 12', 'Np 13', 'Np 14', 'Np 15', 'Np 16', 'Np 17', 'Np 19', 'Np 2', 'Np 20', 'Np 21', 'Np 22', 'Np 23', 'Np 24', 'Np 25', 'Np 26', 'Np 27', 'Np 28', 'Np 29', 'Np 3', 'Np 30', 'Np 31', 'Np 32', 'Np 33', 'Np 34', 'Np 35', 'Np 36', 'Np 37', 'Np 38', 'Np 39', 'Np 4', 'Np 40', 'Np 5', 'Np 6', 'Np 7', 'Np 8', 'Np 9']


In [9]:

transformed_data1 = []
for item in data1:
    
    try:
        for sequence in item[0]:
            transformed_data1.append((sequence, item[1]))
    except:
        pass

In [10]:
transformed_data2 = []
for item in data2:
    
    try:
        for sequence in item[0]:
            transformed_data2.append((sequence, item[1]))
    except:
        pass

In [11]:
import torch
import random

import torch
import random

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, num_channels=6, zero_out_probability=0.5):
        self.data = []
        for sequence, label in data:
            self.data.append((sequence[:6], label))
        self.num_channels = num_channels
        self.zero_out_probability = zero_out_probability

    def __len__(self):
        # Удвоенная длина для оригинальных и аугментированных данных
        return 2 * len(self.data)

    def __getitem__(self, idx):
        sequence, label = self.data[idx // 2]
        sequence_tensor = torch.FloatTensor(sequence)
        label_tensor = torch.tensor(label, dtype=torch.long)

        # Если индекс четный, возвращаем оригинальную последовательность
        if idx % 2 == 0:
            return sequence_tensor, label_tensor

        # Если индекс нечетный, создаем аугментированную последовательность
        augmented_sequence = sequence_tensor.clone()
        for i in range(min(self.num_channels, len(augmented_sequence))):
            if random.random() < self.zero_out_probability:
                augmented_sequence[i] = torch.zeros_like(augmented_sequence[i])

        return augmented_sequence, label_tensor
class DatasetWithoutMasks(torch.utils.data.Dataset):
    def __init__(self, data, num_channels=6, zero_out_probability=0.5):
        self.data = []
        for sequence, label in data:
            self.data.append((sequence[:6], label))
        self.num_channels = num_channels
        self.zero_out_probability = zero_out_probability

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence, label = self.data[idx]
        sequence_tensor = torch.FloatTensor(sequence)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return sequence_tensor, label_tensor

In [12]:
train_dataset = DatasetWithoutMasks(transformed_data1)
validation_dataset = DatasetWithoutMasks(transformed_data2)

In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [14]:
net  = Chrononet()

In [15]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)

In [16]:
def compute_accuracy(predicts, labels):
    return (predicts.argmax(dim=1) == labels).float().mean()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

In [18]:
epochs = 10
best_acc = 0

for epoch in range(epochs):
    running_loss = 0.0
    accuracy = 0.0
    net.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        labels = labels.to(device)
        outputs = net(inputs.to(device))
        loss_value = loss(outputs, labels)
        accuracy += compute_accuracy(outputs, labels)
        loss_value.backward()
        optimizer.step()
        running_loss += loss_value.item()
    print(f'[{epoch + 1}]  Train loss: {running_loss / len(train_loader):.3f}')
    print(f'[{epoch + 1}]  Train accuracy: {accuracy / len(train_loader):.3f}')
    running_loss = 0.0
    accuracy = 0.0
    net.eval()
    with torch.no_grad():
        for i, data in enumerate(validation_loader, 0):
            inputs, labels = data
            labels = labels.to(device)
            inputs = inputs.to(device)
            outputs = net(inputs)
            loss_value = loss(outputs, labels)
            accuracy += compute_accuracy(outputs, labels)
            running_loss += loss_value.item()
    print(f'[{epoch + 1}]  Validation loss: {running_loss / len(validation_loader):.3f}')
    print(f'[{epoch + 1}]  Validation accuracy: {accuracy / len(validation_loader):.3f}')
    if accuracy / len(validation_loader) > best_acc:
        best_acc = accuracy / len(validation_loader)
        torch.save(net.state_dict(), 'best_model_without_mask.pth')
    
    

[1]  Train loss: 0.538
[1]  Train accuracy: 0.677
[1]  Validation loss: 0.504
[1]  Validation accuracy: 0.791
[2]  Train loss: 0.457
[2]  Train accuracy: 0.742
[2]  Validation loss: 0.487
[2]  Validation accuracy: 0.791
[3]  Train loss: 0.437
[3]  Train accuracy: 0.743
[3]  Validation loss: 0.467
[3]  Validation accuracy: 0.790
[4]  Train loss: 0.387
[4]  Train accuracy: 0.780
[4]  Validation loss: 0.304
[4]  Validation accuracy: 0.953
[5]  Train loss: 0.196
[5]  Train accuracy: 0.974
[5]  Validation loss: 0.139
[5]  Validation accuracy: 0.979
[6]  Train loss: 0.106
[6]  Train accuracy: 0.989
[6]  Validation loss: 0.073
[6]  Validation accuracy: 0.992
[7]  Train loss: 0.078
[7]  Train accuracy: 0.992
[7]  Validation loss: 0.064
[7]  Validation accuracy: 0.992
[8]  Train loss: 0.067
[8]  Train accuracy: 0.992
[8]  Validation loss: 0.054
[8]  Validation accuracy: 0.993
[9]  Train loss: 0.063
[9]  Train accuracy: 0.992
[9]  Validation loss: 0.055
[9]  Validation accuracy: 0.992
[10]  Trai

In [19]:
print(len(train_dataset.data))

4518


In [20]:
print(len(validation_dataset.data))

2748


In [21]:
import os
path = 'C:/Users/User/Downloads/sample/Np 1/Nr 1'
data = []
data.append(read_data(path))
transformed_data = []
for item in data:
    try:
        for sequence in item[0]:
            transformed_data.append((sequence, item[1]))
    except:
        pass

C:/Users/User/Downloads/sample/Np 1/Nr 1
C:/Users/User/Downloads/sample/Np 1/Nr 1\N-1.EDF
Extracting EDF parameters from C:\Users\User\Downloads\sample\Np 1\Nr 1\N-1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5818199  =      0.000 ... 29090.995 secs...


C:\Users\User\AppData\Local\Temp\ipykernel_19176\3239762200.py:13: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  datax=mne.io.read_raw_edf(file,preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 661 samples (3.305 s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


Not setting metadata
193 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 193 events and 30000 original time points ...


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.5s finished


0 bad epochs dropped


In [22]:

dataset = CustomDataset(transformed_data)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
model = Chrononet()
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'

In [ ]:
print(torch.Tensor(X_for_cb[0]).shape)

In [ ]:
transformed_data3 = []
for item in X_for_cb:
    
    try:
        for sequence in item:
            print(sequence[:6].shape)
            transformed_data3.append(sequence[:6])
    except:
        pass

In [ ]:
embs_for_cb = []
model.eval()
for s in range(len(transformed_data3)):
    with torch.no_grad():
        embs_for_cb.append(model(torch.Tensor([transformed_data3[s]]).to(device), return_emb=True)[1].cpu().numpy())
    print(s, len(transformed_data3))

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

embs_for_cb = np.array(embs_for_cb).squeeze()
cb_train_X = []
cb_train_y = []
for s in range(len(y_for_cb)):
    if y_for_cb[s] != 0:
        cb_train_X.append(embs_for_cb[s])
        cb_train_y.append(y_for_cb[s])

        
X_train, X_test, y_train, y_test = train_test_split(cb_train_X, cb_train_y, test_size=0.15, random_state=927, shuffle=True)
cb = CatBoostRegressor(loss_function='RMSE', depth=8, iterations=7000, random_seed=927)
cb.fit(embs_for_cb, y_for_cb, eval_set=(X_test, y_test))

In [ ]:
cb_train_y

In [ ]:
from catboost import CatBoostRegressor
arr = []
res = []
cb = CatBoostRegressor().load_model('cb_model.cbm')
with torch.no_grad():
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        labels = labels.to(device)
        inputs = inputs.to(device)
        outputs, emb = model(inputs, return_emb=True)
        res.append(cb.predict(emb.cpu().numpy().squeeze()))
        arr.append(outputs.argmax(dim=1).item())
print(arr.count(0), arr.count(1))
print(np.array(res).mean())

In [ ]:
cb.save_model('cb_model.cbm')